In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Importing Libraries
import pandas as pd
import numpy as np
import io
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from matplotlib import pyplot as plt
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score, confusion_matrix, roc_auc_score, roc_curve, classification_report

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
df=pd.read_csv('/kaggle/input/hr-analytics-job-change-of-data-scientists/aug_train.csv')

In [ ]:
# Heatmap of null values. Too many null values in company_size and company_type
sns.color_palette("Set2")
sns.set_palette("Set2")
sns.heatmap(df.isnull(),vmax=0.6,square=False)

In [ ]:
#Counting Null values
df.isnull().sum()

In [ ]:
df["target"].replace({0: "Not looking for job change", 1: "Looking for job change"}, inplace=True)

In [ ]:

# Relevent Experience vs change of job
df['relevent_experience'].value_counts()
sns.countplot(x=df['relevent_experience'], hue=df['target']) 
#Experience plays a role, People with no relevant experience are most likely to leave

In [ ]:
# Experience vs change of job
plt.figure(figsize=(10,10))
sns.countplot(x=df['experience'], hue=df['target'] ) 
# People with 1 year and less than 1 year experience are more likely to leave.

In [ ]:

# Education Level vs change of job
plt.figure(figsize=(20,10))
ed_level = df.groupby(['target', 'education_level'])['target'].count().unstack('education_level').fillna(0)
ed_level.plot(kind='bar', stacked=True)
plt.xticks(rotation = 0)
ed_level
# Most of the candidates that are looking for chnage of job are Graduates and Post Graduates

In [ ]:
# University enrollement vs change of job
plt.figure(figsize=(20,10))
enrolled_univ = df.groupby(['target', 'enrolled_university'])['target'].count().unstack('enrolled_university').fillna(0)
enrolled_univ.plot(kind='bar', stacked=True)
plt.xticks(rotation = 0)
enrolled_univ
# Full time univerisity enrolled employees have a higher chnace of looking a job change

In [ ]:
# Major Discipline vs change of job
plt.figure(figsize=(20,10))
maj_disc = df.groupby(['target', 'major_discipline'])['target'].count().unstack('major_discipline').fillna(0)
maj_disc.plot(kind='bar', stacked=True)
plt.xticks(rotation = 0)
maj_disc

In [ ]:

df['company_type'].value_counts()
print(df['company_type'].isnull().sum()) 
#More than 25% of the values in this column are null
comp_type = df.groupby(['target', 'company_type'])['target'].count().unstack('company_type').fillna(0)
comp_type.plot(kind='bar', stacked=True)
plt.xticks(rotation = 0)
comp_type

In [ ]:
# Company Size vs change of job
df['company_size'].value_counts() 
print(df['company_size'].isnull().sum()) #almost 25% of the values in this column are null
comp_size = df.groupby(['target', 'company_size'])['target'].count().unstack('company_size').fillna(0)
comp_size.plot(kind='bar', stacked=True)
plt.xticks(rotation = 0)

comp_size

In [ ]:
# checking relations between 'last new job' and 'experience' 
# As candiadates who did not have last new job are having an experience.
df['last_new_job'].value_counts()
never_worked=df['last_new_job']=='never'
df1=df[never_worked]
df1['target'].value_counts()
# 1/3rd of the candidates who never had a last job but have experience in the same company are looking a change of job

In [ ]:
df[never_worked]['experience'].value_counts()
sns.countplot(x=df[never_worked]['experience'], hue=df['target']) 
#Looking to see if candiadates with 'never' in last new job are more likely to stay

In [ ]:
# Experience vs change of job
sns.countplot(x=df['experience'], hue=df['target'] ) 
# Candidates with less experience are likely to look for a new job

In [ ]:
#last new job vs change of job
df['last_new_job'].value_counts()
sns.countplot(x=df['last_new_job'], hue=df['target'])

In [ ]:
# Visualizing Gender
plt.figure(figsize=(30,10))
gender_counts=df['gender'].value_counts()
print(gender_counts)
fig1, ax1 = plt.subplots()
colors = ['lightblue', 'pink', 'grey']
ax1.pie(gender_counts, autopct='%1.1f%%', colors=colors, shadow=True, startangle=90)
ax1.axis('equal') 
plt.show()

In [ ]:
# Most of the candidates in the given data are men
# Gender vs change of job
sns.countplot(x=df['gender'], hue=df['target'])

In [ ]:
sns.displot(df, x="city_development_index", hue="target", kind="kde", fill=True)
# Candidates from lower city development index are more likely to look for change of jobs

In [ ]:
sns.displot(df, x="training_hours", hue="target", kind="kde", fill=True)

In [ ]:
df["target"].replace({ "Not looking for job change":0, "Looking for job change": 1}, inplace=True)

Handling Missing Data Deleting Company_type as it has 32% null values CityNumber(using city development index instead) and Enrollee_id are not very useful for the analysis

In [ ]:
del df['city'] #Considering city_developement_index instead
del df['enrollee_id'] # Not related to attrition rate
del df['company_type'] # Too many null values

In [ ]:
df['target'].value_counts()

In [ ]:
df.head()

In [ ]:
df["last_new_job"].replace({"1": "1", "2": "2", "3":"3", "4":"4", ">4":"5","never":"0" }, inplace=True)
df.last_new_job.value_counts()

In [ ]:
#Coverting last_new_job from category type to numeric type
df["last_new_job"] = pd.to_numeric(df["last_new_job"])

In [ ]:
#Education level is ordinal, hence giving levels 1-5
df["education_level"].replace({"Primary School": "1", "High School": "2", "Graduate":"3", "Masters":"4", "Phd":"5" }, inplace=True)
df.education_level.value_counts()

In [ ]:
#Coverting education_level to category type from object type
df.education_level=df.education_level.astype('category')

In [ ]:
df["company_size"].replace({"<10": 0, "10/49": 1, "50-99":2, "100-500":3, "500-999":4,"1000-4999":5, "5000-9999":6, "10000+":7 }, inplace=True)
df.company_size.value_counts()

In [ ]:
#Coverting company_size to categort type from object type
df.company_size=df.company_size.astype('category')

In [ ]:
# Converting experience to numeric
df["experience"].replace({"<1": 0, ">20": 21}, inplace=True)
df.experience.value_counts()
df["experience"] = pd.to_numeric(df["experience"])

In [ ]:
df.dtypes

In [ ]:
# Drooping records with null values instead of imputing to preserve the logic
df.dropna(inplace=True)

In [ ]:
# Coverting Rest of the Nominal Variables to dummy variables
dfDummies = pd.get_dummies(df, columns=['gender', 'enrolled_university', 'relevent_experience', 'major_discipline'])
dfDummies

In [ ]:
dfDummies.dtypes

In [ ]:
feautures=dfDummies
y=feautures[['target']]

In [ ]:
X=feautures
del X['target']

In [ ]:
num_cols=X.select_dtypes(include=['int64','float64']).columns.to_list()
num_cols

In [ ]:
X.dtypes

In [ ]:
# Success class is Imbalanced data
y.value_counts()

In [ ]:
y_arr= np.array(y)
X_arr=np.array(X)

In [ ]:
from imblearn.over_sampling import SMOTE
X_train, X_test, y_train, y_test = train_test_split(X_arr, y_arr, test_size = 0.2, random_state = 2)
smote = SMOTE(random_state=2)
# Using Over-sampling Technique - one of the most commonly used resampling 
# techniques to solve the imbalance problem.
X_train_balanced, Y_train_balanced = smote.fit_resample(X_train, y_train)

In [ ]:
X_train_balanced

In [ ]:
#Dummy Classifier
from sklearn.dummy import DummyClassifier
NR = DummyClassifier(strategy= 'most_frequent').fit(X_train_balanced, Y_train_balanced)
y_pred = NR.predict(X_test)

# Model Evaluation metrics of Naive rule
print('Accuracy Score : ' + str(accuracy_score(y_test,y_pred)))
print('Precision Score : ' + str(precision_score(y_test,y_pred)))
print('Recall Score : ' + str(recall_score(y_test,y_pred)))
print('F1 Score : ' + str(f1_score(y_test,y_pred)))

#Dummy Classifier Confusion matrix
from sklearn import metrics
print('Confusion Matrix : \n' + str(confusion_matrix(y_test,y_pred)))
fpr, tpr, thresholds = metrics.roc_curve(y_test,y_pred)
metrics.auc(fpr, tpr)


Naive rule classifies all records as the most frequently occuring class, in this case class 0, not looking for job change. Though the accuracy is above 80%, the classification is poor. AUC is always 50%

In [ ]:
# Using Decision Trees without any hypertuning
model = DecisionTreeClassifier(criterion='entropy')
model # it shows the default parameters

In [ ]:
#Fitting model
model.fit(X_train_balanced, Y_train_balanced)

In [ ]:
#Performing Prediction
predTree = model.predict(X_test)

In [ ]:
#Calculating metrics
print(classification_report(y_test,predTree))
# Model Evaluation metrics of Decision Tree
print('Accuracy Score : ' + str(accuracy_score(y_test,predTree)))
print('Precision Score : ' + str(precision_score(y_test,predTree)))
print('Recall Score : ' + str(recall_score(y_test,predTree)))
print('F1 Score : ' + str(f1_score(y_test,predTree)))
#Decision Tree Confusion matrix
print('Confusion Matrix : \n' + str(confusion_matrix(y_test,predTree)))


In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(y_test,predTree)
metrics.auc(fpr, tpr)

In [ ]:
path=model.cost_complexity_pruning_path(X_train_balanced,Y_train_balanced)
alphas=path['ccp_alphas']
alphas
accuracy_train,accuracy_test=[],[]
for i in alphas:
  tree=DecisionTreeClassifier(ccp_alpha=i)
  tree.fit(X_train_balanced,Y_train_balanced)
  y_train_pred=tree.predict(X_train_balanced)
  y_test_pred=tree.predict(X_test)
  accuracy_train.append(accuracy_score(Y_train_balanced, y_train_pred))
  accuracy_test.append(accuracy_score(y_test, y_test_pred))

In [ ]:
plt.figure(figsize=(14,7))
sns.lineplot(y=accuracy_train, x=alphas, label='Train Accuracy')
sns.lineplot(y=accuracy_test, x=alphas, label='Test Accuracy')
#plt.xticks(ticks=np.arange(0, 0.5 ,1))
plt.xlabel('cost complexity') # Label x-axis
plt.ylabel('Accuracy') # Label y-axis
plt.title("ccp vs accuracy")
plt.show()

In [ ]:
# Modeling Using Decision Trees
tree2=DecisionTreeClassifier(ccp_alpha=0.025, random_state=0)
tree2.fit(X_train_balanced, Y_train_balanced)
pred=tree2.predict(X_test)
print(accuracy_score(y_test, pred))
#Accuracy has increased in predicting test set after implementing cost complexity

In [ ]:
'''Model Evaluation metrics of Decision Tree after hypertuning with ccp'''
print('Accuracy Score : ' + str(accuracy_score(y_test,pred)))
print('Precision Score : ' + str(precision_score(y_test,pred)))
print('Recall Score : ' + str(recall_score(y_test,pred)))
print('F1 Score : ' + str(f1_score(y_test,pred)))
#Decision Tree Confusion matrix
print('Confusion Matrix : \n' + str(confusion_matrix(y_test,pred)))
print('AUC : ' + str(metrics.auc(fpr, tpr)))

In [ ]:
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(y_test,pred)
metrics.auc(fpr, tpr)
print('AUC : ' + str(metrics.auc(fpr, tpr)))
print(classification_report(y_test, pred))

In [ ]:
data_feature_names=['city_development_index', 'education_level', 'experience',
       'company_size', 'last_new_job', 'training_hours', 'gender_Female',
       'gender_Male', 'gender_Other', 'enrolled_university_Full time course',
       'enrolled_university_Part time course',
       'enrolled_university_no_enrollment',
       'relevent_experience_Has relevent experience',
       'relevent_experience_No relevent experience', 'major_discipline_Arts',
       'major_discipline_Business Degree', 'major_discipline_Humanities',
       'major_discipline_No Major', 'major_discipline_Other',
       'major_discipline_STEM']

Random Forests

In [ ]:
re=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=10, max_features='auto',
                       min_samples_leaf=1, min_samples_split=2,n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
# Fitting the model
re.fit(X_train_balanced, Y_train_balanced)

In [ ]:
# Predicting Values
y_hat=re.predict(X_test)

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(y_test,y_hat)
metrics.auc(fpr, tpr)
print('AUC : ' + str(metrics.auc(fpr, tpr)))

In [ ]:
print('Accuracy Score : ' + str(accuracy_score(y_test,y_hat)))
print('Precision Score : ' + str(precision_score(y_test,y_hat)))
print('Recall Score : ' + str(recall_score(y_test,y_hat)))
print('F1 Score : ' + str(f1_score(y_test,y_hat)))
print('Confusion Matrix : \n' + str(confusion_matrix(y_test,y_hat)))
print('AUC : ' + str(metrics.auc(fpr, tpr)))

In [ ]:
plt.plot(fpr, tpr)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

max_depth_list = [1,2,3,4,5,6,7,8,9,10]

train_errors = [] # Log training errors for each model
test_errors = [] # Log testing errors for each model

for x in max_depth_list:
    rf = RandomForestClassifier(max_depth=x) 
    rf.fit(X_train_balanced,Y_train_balanced)
    train_z = rf.predict(X_train_balanced)
    test_z = rf.predict(X_test)
    #train_errors.append(accuracy_score(X_train_balanced, train_z))
    test_errors.append(accuracy_score(y_test, test_z))
x = np.arange(len(max_depth_list)) + 1 # Create domain for plot
 # Plot training error over domain
plt.plot(x, test_errors, label='Testing Error') # Plot testing error over domain
plt.xlabel('Maximum Depth') # Label x-axis
plt.ylabel('Total Error') # Label y-axis
plt.legend() # Show plot labels as legend
plt.plot() # Show graph

In [ ]:
rfc=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=3, max_features='auto',
                       max_leaf_nodes=None, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
# Hypertuning Max_depth
rfc.fit(X_train_balanced, Y_train_balanced)
y_hat1=rfc.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score
print('Accuracy Score : ' + str(accuracy_score(y_test,y_hat1)))
print('Precision Score : ' + str(precision_score(y_test,y_hat1)))
print('Recall Score : ' + str(recall_score(y_test,y_hat1)))
print('F1 Score : ' + str(f1_score(y_test,y_hat1)))

from sklearn.metrics import confusion_matrix
print('Confusion Matrix : \n' + str(confusion_matrix(y_test,y_hat1)))
print('AUC : ' + str(metrics.auc(fpr, tpr)))

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(y_test,y_hat1)
print(metrics.auc(fpr, tpr))
plt.plot(fpr, tpr)

In [ ]:
# Using GridSearch
from sklearn.datasets import make_blobs
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
n_estimators = [100]
max_depth = [3, 5, 8]
min_samples_split = [2, 5, 10, 15, 100]
min_samples_leaf = [1, 2, 5, 10] 

hyperF = dict(n_estimators = n_estimators, max_depth = max_depth,  
              min_samples_split = min_samples_split, 
             min_samples_leaf = min_samples_leaf)

gridF = GridSearchCV(rfc, hyperF, cv = 3, verbose = 0, 
                      n_jobs = -1)
bestF = gridF.fit(X_train_balanced, Y_train_balanced)

In [ ]:
print("Best: %f using %s" % (bestF.best_score_, bestF.best_params_))
means = bestF.cv_results_['mean_test_score']
stds = bestF.cv_results_['std_test_score']
params = bestF.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
#Based on gridsearch
rfc1=RandomForestClassifier(bootstrap=True,max_depth=3, min_samples_leaf=1, min_samples_split=2, n_estimators=500) 

In [ ]:
rfc1.fit(X_train_balanced, Y_train_balanced)
y_hat2=rfc1.predict(X_test)

In [ ]:
print('Accuracy Score : ' + str(accuracy_score(y_test,y_hat2)))
print('Precision Score : ' + str(precision_score(y_test,y_hat2)))
print('Recall Score : ' + str(recall_score(y_test,y_hat2)))
print('F1 Score : ' + str(f1_score(y_test,y_hat2)))

print('Confusion Matrix : \n' + str(confusion_matrix(y_test,y_hat2)))
#print('AUC : ' + str(metrics.auc(fpr, tpr)))

Logistic Regresssion

In [ ]:
# Converting m dummies to m-1 dummies to prevent multicollinearity
X=feautures
del X['gender_Other']
del X['enrolled_university_no_enrollment']
del X['relevent_experience_Has relevent experience']
del X['major_discipline_Other']

In [ ]:
num_cols

In [ ]:
x=X[['city_development_index', 'education_level', 'experience',
       'company_size', 'last_new_job', 'training_hours', 'gender_Female',
       'gender_Male', 'enrolled_university_Full time course',
       'enrolled_university_Part time course',
       'relevent_experience_No relevent experience', 'major_discipline_Arts',
       'major_discipline_Business Degree', 'major_discipline_Humanities',
       'major_discipline_No Major', 'major_discipline_STEM']].corr()
ax = sns.heatmap(x, cmap="RdYlBu")
# turn the axis label
for item in ax.get_yticklabels():
    item.set_rotation(0)

for item in ax.get_xticklabels():
    item.set_rotation(90)

# save figure
plt.savefig('seabornPandas.png', dpi=100)
plt.show(30,30)

In [ ]:
#Standardising the numeric variables except categorical and dummy variables
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X[num_cols] = sc.fit_transform(X[num_cols])
X.head()

In [ ]:
X_arr= np.array(X)
y_arr= np.array(y)

In [ ]:
y_arr

In [ ]:
from imblearn.over_sampling import SMOTE
X_train, X_test, y_train, y_test = train_test_split(X_arr, y_arr, test_size = 0.2, random_state = 2)
smote = SMOTE()
# Using Over-sampling Technique - one of the most commonly used resampling 
# techniques to solve the imbalance problem.
X_train_balanced, Y_train_balanced = smote.fit_resample(X_train, y_train)

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(penalty="l1", C=1e42, solver='liblinear')
lr.fit(X_train_balanced, Y_train_balanced)
print('intercept ', lr.intercept_[0])
coef=pd.DataFrame({'coeff': (lr.coef_[0])}, index=X.columns)
coef.sort_values(by='coeff', ascending=True)

Candidates with more experience are less likely to leave

STEM, Humanities are more likely to leave

Females are more likely to leave compared to males

Candidates enrolled in full time are more likely to quit

In [ ]:
y_hat = lr.predict(X_test)
lr_proba = lr.predict_proba(X_test)
lr_proba

In [ ]:
print('Accuracy Score : ' + str(accuracy_score(y_test,y_hat)))
print('Precision Score : ' + str(precision_score(y_test,y_hat)))
print('Recall Score : ' + str(recall_score(y_test,y_hat)))
print('F1 Score : ' + str(f1_score(y_test,y_hat)))

print('Confusion Matrix : \n' + str(confusion_matrix(y_test,y_hat)))

In [ ]:
import itertools

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=0)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        #print("Normalized confusion matrix")
    else:
        1#print('Confusion matrix, without normalization')

    #print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
cnf_matrix = confusion_matrix(y_test, y_hat)
print("Recall metric in the testing dataset: {}%".format(100*cnf_matrix[1,1]/(cnf_matrix[1,0]+cnf_matrix[1,1])))
#print("Precision metric in the testing dataset: {}%".format(100*cnf_matrix[0,0]/(cnf_matrix[0,0]+cnf_matrix[1,0])))
# Plot non-normalized confusion matrix
class_names = [0,1]
plt.figure()
plot_confusion_matrix(cnf_matrix , classes=class_names, title='Confusion matrix')
plt.show()

In [ ]:
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(y_test,y_hat)
metrics.auc(fpr, tpr)

In [ ]:
roc_auc=roc_auc_score(y_test, y_hat)
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b',label='AUC = %0.3f'% roc_auc)
plt.legend(loc='lower right')
plt.plot([0,1],[0,1],'r--')
plt.xlim([-0.1,1.0])
plt.ylim([-0.1,1.01])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
# Using Stats model to identify important features (profiling)
import statsmodels.api as sm
X_train_balanced=pd.DataFrame(X_train_balanced)
X_train_balanced=X_train_balanced.rename(columns={0: 'city_development_index', 1: 'education_level', 2:'experience',
       3:'company_size', 4:'last_new_job', 5:'training_hours', 6:'gender_Female',
       7:'gender_Male', 8:'enrolled_university_Full time course',
       9:'enrolled_university_Part time course',
       10:'relevent_experience_No relevent experience', 11:'major_discipline_Arts',
       12:'major_discipline_Business Degree', 13:'major_discipline_Humanities',
      14: 'major_discipline_No Major', 15:'major_discipline_STEM'})
log_reg = sm.Logit(Y_train_balanced, X_train_balanced).fit()

In [ ]:
# printing the summary table
print(log_reg.summary())

In [ ]:
# Altering cut-off value to classify better
# changing cut off value from default to
prob1=lr.predict_proba(X_test)[:,1]
predicted=[1 if i > 0.4 else 0 for i in prob1]
print('Confusion Matrix : \n' + str(confusion_matrix(y_test,predicted)))
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(y_test,predicted)
metrics.auc(fpr, tpr)

In [ ]:
print('Accuracy Score : ' + str(accuracy_score(y_test,predicted)))
print('Precision Score : ' + str(precision_score(y_test,predicted)))
print('Recall Score : ' + str(recall_score(y_test,predicted)))
print('F1 Score : ' + str(f1_score(y_test,predicted)))

As mentioned earlier, the class distribution is skewed in this data set. Hence, the accuracy metric is biased and not preferable. Hence, primarily looked at the recall rates of different models to evaluate the performance.

Comparing the recall rates of models, almost all the models performed similarly but good classification and quantification of the predictors were obtained from Logistic Regression Model. Coming to the profiling, both the import features from decision trees and logistic regression point to city development index and relevant experience. However, this information was more interpretable in logistic regression summary than in trees.

Conclusion
As a result of this analysis, I have come to know that Candidates coming from cities with lesser development indexes are more likely to find new employment. Experience plays an important role in classifying these candidates. Candidates that either have relevant experience or experience at a previous company or overall experience above 5 years are less probable to leave. Candidates that have a major discipline in the STEM field are more likely to leave than other disciplines. Education plays a moderately important role in classifying candidates.